In [59]:
import pandas as pd 
import numpy as np
import csv
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [66]:
def getXY(filename):
  X = pd.read_csv(filename)
  if(filename=='merged.csv'):
        X = X.iloc[:,564:]

  x_drop = ["LONGITUDE", "LATITUDE", "FLOOR", "BUILDINGID", "SPACEID", "RELATIVEPOSITION", "USERID", "PHONEID", "TIMESTAMP"]
  y_cols = ["LONGITUDE", "LATITUDE","FLOOR","BUILDINGID"]

  x_cols = list(set(X.columns)-set(x_drop))
  X_train = X[x_cols]
  X_train.iloc[:, 0:520] = np.where(X_train.iloc[:, 0:520] <= 0, X_train.iloc[:, 0:520] + 105, X_train.iloc[:, 0:520] - 100)

  Y_train = X[y_cols]
  print(Y_train["LONGITUDE"].min())
  print(Y_train["LATITUDE"].min())

  Y_train["LONGITUDE"] += 7700
  Y_train["LATITUDE"] -= 4864745

  return X_train, Y_train

#X_train, Y_train = getXY('TrainingData.csv')
#print(X_train)
#print(Y_train)

full_input = pd.read_csv('merged.csv')
act_input = full_input.iloc[:,1:563]
act_true = full_input.iloc[:,563]

loc_input, loc_true = getXY('merged.csv')
print(X_test)
print(Y_test)

-7691.109200000763
4864754.248742422
     WAP379  WAP029  WAP230  WAP232  WAP009  WAP227  WAP113  WAP240  WAP475  \
0         0       0       0       0       0       0       0       0       0   
1         0       0       0       0       0       0       0       0       0   
2         0       0       0       0       0       0       0       0       0   
3         0       0       0       0       0       0       0       0       0   
4         0      43       0       0       0       0       0       0       0   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
195       0      12       0       0       0       0       0       0       0   
196       0       0       0       0       0       0       0       0       0   
197       0       0       0       0       0       0       0       0       0   
198       0       0       0       0       0       0       0       0       0   
199       0       0       0       0       0       0       0       0       0   

     WAP006  .

In [54]:
X,Y = getXY('ValidationData.csv')


-7695.93875492993
4864748.01548531


In [55]:
def pr(x):
    for i in range(len(x)):
        if x[i]!=0:
            print(i,x[i])
pr(X.iloc[0,:])
pr(X_test.iloc[0,:])

376 14
376 14


In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier as rf
import pickle


y_cols = ["LONGITUDE", "LATITUDE","FLOOR","BUILDINGID"]
longitude_model = RandomForestRegressor()
longitude_model.fit(X_train, Y_train[y_cols[0]])
pickle.dump(longitude_model, open('longitude_model.sav', 'wb'))

latitude_model = RandomForestRegressor()
latitude_model.fit(X_train, Y_train[y_cols[1]])
pickle.dump(latitude_model, open('latitude_model.sav', 'wb'))

floor_model = rf()
floor_model.fit(X_train, Y_train[y_cols[2]])
pickle.dump(floor_model, open('floor_model.sav', 'wb'))

building_model = rf()
building_model.fit(X_train, Y_train[y_cols[3]])
pickle.dump(building_model, open('building_model.sav', 'wb'))

'''

'''

"\n\nactivity_model = pickle.load(open('activity_model.sav', 'rb'))\nlatitude_model = pickle.load(open('latitude_model.sav', 'rb'))\nlongitude_model = pickle.load(open('longitude_model.sav', 'rb'))\nfloor_model = pickle.load(open('floor_model.sav', 'rb'))\nbuilding_model = pickle.load(open('building_model.sav', 'rb'))\n\n"

In [61]:

activity_model = 0
latitude_model = 0
longitude_model = 0
floor_model = 0
building_model = 0


activity_model = pickle.load(open('activity_model.sav', 'rb'))
latitude_model = pickle.load(open('latitude_model.sav', 'rb'))
longitude_model = pickle.load(open('longitude_model.sav', 'rb'))
floor_model = pickle.load(open('floor_model.sav', 'rb'))
building_model = pickle.load(open('building_model.sav', 'rb'))


In [83]:
def predict(user_id):
    pred = [user_id]
    activity_input = (np.array(act_input.iloc[user_id,:])).reshape(1,-1)
    pred.append(activity_model.predict(activity_input))
    
    localisation_input = (np.array(loc_input.iloc[user_id,:])).reshape(1,-1)
    pred.append(floor_model.predict(localisation_input))
    pred.append(building_model.predict(localisation_input))
    pred.append(latitude_model.predict(localisation_input))
    pred.append(longitude_model.predict(localisation_input))
    
    print("User: {} {} on Floor: {} in Building: {} Location Coordinates : {},{}".format(pred[0], pred[1][0],pred[2][0],pred[3][0],pred[4][0],pred[5][0])) 
    print("G.Truth {} on Floor: {} in Building: {} Location Coordinates : {},{}\n".format(act_true[user_id],loc_true.iloc[user_id,2],loc_true.iloc[user_id,3],loc_true.iloc[user_id,1],loc_true.iloc[user_id,0])) 
    # print(pred)
    return pred

    

In [84]:

loc_true = Y_test
fa = 0
ba = 0
#n = len(X_test)
n=15
for i in range(0,n):
    pred = predict(i)
    if(pred[2] == loc_true.iloc[i,2]):
        fa += 1/n
    if(pred[3] == loc_true.iloc[i,3]):
        ba += 1/n
    
print(fa)
print(ba)

User: 0 STANDING on Floor: 0 in Building: 1 Location Coordinates : 116.5110549105443,258.7414541684134
G.Truth STANDING on Floor: 1 in Building: 1 Location Coordinates : 144.66291668452322,184.08320060014103

User: 1 STANDING on Floor: 4 in Building: 2 Location Coordinates : 93.55783971042372,268.84989603394723
G.Truth STANDING on Floor: 4 in Building: 2 Location Coordinates : 94.7403071783483,316.1327790177884

User: 2 STANDING on Floor: 4 in Building: 2 Location Coordinates : 99.04900242491658,293.8436690032829
G.Truth STANDING on Floor: 4 in Building: 2 Location Coordinates : 101.53372230008245,325.6979200593696

User: 3 STANDING on Floor: 4 in Building: 2 Location Coordinates : 106.70595621164888,282.3626544030852
G.Truth STANDING on Floor: 4 in Building: 2 Location Coordinates : 97.82925374154001,334.17511744946114

User: 4 STANDING on Floor: 2 in Building: 0 Location Coordinates : 175.1487485679984,54.17249150913212
G.Truth STANDING on Floor: 2 in Building: 0 Location Coordinates

In [85]:
def read_user_id():
    print("Enter User Id b'n 0-199:")
    user_id = int(input())
    while user_id<0 or user_id>=200:
        print("Enter User Id b'n 0-199:")
        user_id = int(input())
    return user_id



In [87]:
predict(read_user_id())

Enter User Id b'n 0-199:
5
User: 5 STANDING on Floor: 2 in Building: 2 Location Coordinates : 79.451486806795,362.5132080000639
G.Truth STANDING on Floor: 2 in Building: 2 Location Coordinates : 80.46686277911067,361.19278958122595



[5,
 array(['STANDING'], dtype=object),
 array([2]),
 array([2]),
 array([79.45148681]),
 array([362.513208])]